### LSTM으로 삼성전자 주식 예측하기

라이브러리 설치

In [ ]:
!pip install finance-datareader

In [ ]:
import FinanceDataReader as fdr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

데이터셋 로드

In [ ]:
samsung = fdr.DataReader('005930', '2016')
print(samsung)

openValues = samsung[['Open']]  # 시작값만 사용

특징값 정규화

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))   # 0에서 1사이의 값으로 변환
scaled = scaler.fit_transform(openValues)
print(scaled)

훈련 데이터와 테스트 데이터의 분리

In [ ]:
TEST_SIZE = 200
train_data = scaled[:-TEST_SIZE]
test_data = scaled[-TEST_SIZE:]

순차 훈련 데이터 생성. 30개씩 분리

In [ ]:
def make_sample(data, window):
    train = []
    target = []
    for i in range(len(data)-window):
        train.append(data[i:i+window])
        target.append(data[i+window])
    return np.array(train), np.array(target)

X_train, y_train = make_sample(train_data, 30)

LSTM 구축

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(LSTM(16,      # 셀 갯수 = 16
                input_shape=(X_train.shape[1], 1),  # 훈련 데이터의 두번째 차원을 입력으로
                activation='tanh', 
                return_sequences=False)     # 출력 시퀀스의 마지막 출력만을 반환
          )
model.add(Dense(1))
model.summary()

모델 학습

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_train, y_train, epochs = 100, batch_size = 16)

데스트셋을 이용하여 미래 주가 에측

In [ ]:
X_test, y_test = make_sample(test_data, 30)
pred = model.predict(X_test)

그래프 그리기. 200개의 테스트 데이터를 윈도우 크기=30으로 분석, 예측한 결과

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 9))
plt.plot(y_test, label='stock price')
plt.plot(pred, label='predicted stock price')
plt.legend()
plt.show()